# What is Sharding?

# When to Shard

# Sharding Architecture

<img src="31.png">

<img src="32.png">

<img src="33.png">

<img src="34.png">

<img src="35.png">

<img src="36.png">

<img src="37.png">

<img src="38.png">

---

# Setting Up a Sharded Cluster

In [ ]:
sharding:
  clusterRole: configsvr
replication:
  replSetName: m103-csrs
security:
  keyFile: /var/mongodb/pki/m103-keyfile
net:
  bindIp: localhost,192.168.103.100
  port: 26001
systemLog:
  destination: file
  path: /var/mongodb/db/csrs1.log
  logAppend: true
processManagement:
  fork: true
storage:
  dbPath: /var/mongodb/db/csrs1

In [ ]:
sharding:
  clusterRole: configsvr
replication:
  replSetName: m103-csrs
security:
  keyFile: /var/mongodb/pki/m103-keyfile
net:
  bindIp: localhost,192.168.103.100
  port: 26002
systemLog:
  destination: file
  path: /var/mongodb/db/csrs2.log
  logAppend: true
processManagement:
  fork: true
storage:
  dbPath: /var/mongodb/db/csrs2

In [ ]:
sharding:
  clusterRole: configsvr
replication:
  replSetName: m103-csrs
security:
  keyFile: /var/mongodb/pki/m103-keyfile
net:
  bindIp: localhost,192.168.103.100
  port: 26003
systemLog:
  destination: file
  path: /var/mongodb/db/csrs3.log
  logAppend: true
processManagement:
  fork: true
storage:
  dbPath: /var/mongodb/db/csrs3

In [ ]:
mongod -f csrs_1.conf
mongod -f csrs_2.conf
mongod -f csrs_3.conf

In [ ]:
mongo --port 26001

In [ ]:
rs.initiate()

In [ ]:
use admin
db.createUser({
  user: "m103-admin",
  pwd: "m103-pass",
  roles: [
    {role: "root", db: "admin"}
  ]
})

In [ ]:
db.auth("m103-admin", "m103-pass")

In [ ]:
rs.add("192.168.103.100:26002")
rs.add("192.168.103.100:26003")

In [ ]:
rs.isMaster()

sharding:
  configDB: m103-csrs/192.168.103.100:26001,192.168.103.100:26002,192.168.103.100:26003
security:
  keyFile: /var/mongodb/pki/m103-keyfile
net:
  bindIp: localhost,192.168.103.100
  port: 26000
systemLog:
  destination: file
  path: /var/mongodb/db/mongos.log
  logAppend: true
processManagement:
  fork: true

In [ ]:
mongos -f mongos.conf

In [ ]:
mongo --port 26000 --username m103-admin --password m103-pass --authenticationDatabase admin

In [ ]:
sh.status()

In [ ]:
sharding:
  clusterRole: shardsvr
storage:
  dbPath: /var/mongodb/db/node1
  wiredTiger:
    engineConfig:
      cacheSizeGB: .1
net:
  bindIp: 192.168.103.100,localhost
  port: 27011
security:
  keyFile: /var/mongodb/pki/m103-keyfile
systemLog:
  destination: file
  path: /var/mongodb/db/node1/mongod.log
  logAppend: true
processManagement:
  fork: true
replication:
  replSetName: m103-repl

In [ ]:
sharding:
  clusterRole: shardsvr
storage:
  dbPath: /var/mongodb/db/node2
  wiredTiger:
    engineConfig:
      cacheSizeGB: .1
net:
  bindIp: 192.168.103.100,localhost
  port: 27012
security:
  keyFile: /var/mongodb/pki/m103-keyfile
systemLog:
  destination: file
  path: /var/mongodb/db/node2/mongod.log
  logAppend: true
processManagement:
  fork: true
replication:
  replSetName: m103-repl

In [ ]:
sharding:
  clusterRole: shardsvr
storage:
  dbPath: /var/mongodb/db/node3
  wiredTiger:
    engineConfig:
      cacheSizeGB: .1
net:
  bindIp: 192.168.103.100,localhost
  port: 27013
security:
  keyFile: /var/mongodb/pki/m103-keyfile
systemLog:
  destination: file
  path: /var/mongodb/db/node3/mongod.log
  logAppend: true
processManagement:
  fork: true
replication:
  replSetName: m103-repl

In [ ]:
mongo --port 27012 -u "m103-admin" -p "m103-pass" --authenticationDatabase "admin"

In [ ]:
use admin
db.shutdownServer()

In [ ]:
mongod -f node2.conf

In [ ]:
# Do the same thing for the third node
mongo --port 27013 -u "m103-admin" -p "m103-pass" --authenticationDatabase "admin"

In [ ]:
use admin
db.shutdownServer()

mongod -f node3.conf

In [ ]:
mongo --port 27011 -u "m103-admin" -p "m103-pass" --authenticationDatabase "admin"

In [ ]:
use admin
rs.stepDown()

In [ ]:
db.shutdownServer()

In [ ]:
mongod -f node1.conf

In [ ]:
# connect back to mongos
mongo --port 26000 --username m103-admin --password m103-pass --authenticationDatabase admin

In [ ]:
sh.addShard("m103-repl/192.168.103.100:27012")

In [ ]:
sh.status()

# Config DB

In [ ]:
sh.status()

In [ ]:
use config

In [ ]:
show collections

In [ ]:
# 先看看 databases 這個 collection
db.databases.find().pretty() 

<img src="20.PNG">

In [ ]:
db.collections.find().prtty()

<img src="21.PNG">

In [ ]:
db.shards.find().pretty()

<img src="22.PNG">

In [ ]:
db.chunks.find().pretty()

<img src="23.PNG">

<img src="24.PNG">

In [ ]:
db.mongos.find().pretty()

<img src="25.PNG">

---

# Shard Keys

<img src="26.PNG">

### How To Shard

### 範例01

In [ ]:
show dbs
sh.status()

In [ ]:
use m103
show collections

In [ ]:
Enable sharding on the m103 database:

In [ ]:
sh.enableSharding("m103")

In [ ]:
db.products.findOne()

In [ ]:
db.products.createIndex({ "sku": 1 })

In [ ]:
sh.shardCollection( "m103.products", { "sku": 1 })

In [ ]:
sh.status()

<img src="27.PNG">

# Picking a Good Shard Key

### read isolation

# Hashed Shard Keys

### Hashed Sharding Shard Key

### Hashed vs Ranged Sharding

<img src="sharded-cluster-monotonic-distribution.bakedsvg.svg">

<img src="sharded-cluster-hashed-distribution.bakedsvg.svg">

### Hashed Shard Key Considerations

### Sharding using a Hashed Shard Key

# Chunks

In [ ]:
show dbs
use config
show collections

In [ ]:
# Find one document from the chunks collection:
db.chunks.findOne()

<img src="28.png">

### ChunkSize

In [ ]:
# 先看看我們現在有幾個 Chunks
sh.status()

In [ ]:
# value 的單位是 MB
use config
db.settings.save({_id: "chunksize", value: 2}) 

In [ ]:
sh.status()

In [ ]:
mongoimport /dataset/products.part2.json --port 26000 -u "m103-admin" -p "m103-pass" --authenticationDatabase "admin" --db m103 --collection products

In [ ]:
sh.status()

### Jumbo Chunks

---

# Balancing

In [ ]:
sh.startBalancer(timeout, interval)

In [ ]:
sh.stopBalancer(timeout, interval)

In [ ]:
sh.setBalancerState(boolean)

# Queries in a Sharded Cluster

<img src="29.PNG">

<img src="30.PNG">

### Sort, Limit, Skip in sharded clusters

# Targeted Queries vs Scatter Gather

In [ ]:
db.products.find( { "sku": 20009151 } )

In [ ]:
db.products.find( { "type": "movie" } )

In [ ]:
# one thing to emphasize, if you're using a compound index as your 
# shard key, then you can specify each prefix up to the entire shard key and still get a targeted query.
db.products.find( { "sku": ... } )
db.products.find( { "sku": ... , "type": ... } )
db.products.find( { "sku": ... , "type": ... , "name": ... } )

In [ ]:
db.products.find( { "type": ... } )
db.products.find( { "name": ... } )

---

In [ ]:
use m103
show collections

In [ ]:
# 跟下面得到一樣的結果，但是上面這個 query 快很多
db.products.find({"sku" : 1000000749 }).explain()

In [ ]:
db.products.find( {
  "name" : "Gods And Heroes: Rome Rising - Windows [Digital Download]" }
).explain()